# Selenium scraper for multiple websites which let you search by ZIP

### pip installs libraries.
note: %%capture suppresses output

In [2]:
%%capture
pip install selenium

In [9]:
%%capture
pip install webdriver-manager

## Define parameters.

In [10]:
STATE_TO_SEARCH = None #'CO'
    # Options include 'CO' and None
SITES_TO_SEARCH = ['Rent.']
    # Options include 'Realtor' and 'Rent.'

WAIT_TIME_DICT = {'CONSTANT':{'Realtor':10,'Rent.':0,'other': 2},
                     # whether or not site redirects to CAPTCHA
                  'FAILURE' :{'Realtor':25,'Rent.':5,'other':10}
                     # after site redirects to CAPTCHA
            }

## Import libraries.

In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager 
    #from webdriver_manager.firefox import GeckoDriverManager
import pandas # for reading the txt with ZIPs
import datetime # for file names
import os # for checking if files exist
import bs4 # for knowing how many pages there are per ZIP
from time import sleep
from tqdm import tqdm # progress bar
import requests # for headless version
Service = ChromeService

In [12]:
DF = pandas.read_csv("sorted_zipmha22.txt",
                     delimiter = " ", names = ["ZIP","MHA"])
DF['State'] = DF.apply(lambda row: row['MHA'][:2], axis = 1)
if STATE_TO_SEARCH:
    assert type(STATE_TO_SEARCH) == str
    DF = DF[DF['State'] == STATE_TO_SEARCH]
ZIPs = list(DF['ZIP'])
ZIPs = [str(ZIP).zfill(5) for ZIP in ZIPs]
print("There are",len(ZIPs),"ZIPs to scrape")

There are 41070 ZIPs to scrape


In [13]:
PATH = ChromeDriverManager().install()
#PATH = GeckoDriverManager().install()

## Define functions.

In [14]:
def WAIT_TIME(context, site):
    if site in WAIT_TIME_DICT[context].keys():
        return WAIT_TIME_DICT[context][site]
    else:
        return WAIT_TIME_DICT[context]['other']

In [15]:
# /appartments includes houses for rent
def URL(site, ZIP, page = 1):
    assert type(site) == str
    assert type(ZIP ) == str
    return {'Realtor':"https://www.realtor.com/apartments/{}/pg-{}".format(ZIP, page),
            'Rent.'  :"https://www.rent.com/zip-{}-apartments?page={}".format(ZIP, page),
            #'Zillow' :"https://www.zillow.com/homes/{}_r/".format(ZIP),
            'trulia' :"https://www.trulia.com/for_rent/{}_zip/".format(ZIP),
            'REDFIN' :"https://www.redfin.com/zipcode/{}/apartments-for-rent/page-{}".format(ZIP, page)
           }[site]

In [16]:
def generateFilePath(site, ZIP, page = '1'):
    for var in [site, ZIP, page]: assert type(var) == str
    fileName = page.zfill(2)
    if not ('.' in fileName): fileName += '.html'
    
    filePathParts = ['HTMLs/']

    if STATE_TO_SEARCH: # if searching just one state
        assert type(STATE_TO_SEARCH) == str
        filePathParts.append(STATE_TO_SEARCH + '/')
    
    filePathParts.append(site.replace('.','') + '/')
    
    dateString = str(datetime.date.today())
    filePathParts.append(dateString + '/')
    
    filePathParts.append(ZIP + '/')
    
    filePath = ""
    for filePathPart in filePathParts:
        filePath += filePathPart
        if not os.path.exists(filePath): os.mkdir(filePath)
    
    filePath += fileName
    
    return filePath

In [17]:
def HTML_to_file(HTML, filePath):
    # Does not check if file already exists.
        # Only run if file does not exists or should be written over.
    with open(filePath, 'w', encoding="utf-8") as f:
        f.write(HTML)
        f.close()

In [18]:
HTML_MIN_SIZE = 100 # KB
HTML_MIN_SIZE *= 1000 #KB -> B

def isValidHTML(filePath):
    if os.path.exists(filePath):
        if os.path.getsize(filePath) < HTML_MIN_SIZE:
            return False
        else:
            return True
    else:
        return False

In [19]:
def parseNumPages(site, HTML):
    if site == 'Rent.':
        PROPS_PER_PAGE = 30
        numPropsTags = bs4.BeautifulSoup(HTML).find_all(class_ = "truncate text-gray-600")
        if len(numPropsTags) >= 1:
            numPropsText = numPropsTags[0].text
            numPropsText = numPropsText.split(" ")[0] # take only text before space
            numPropsText = numPropsText.replace(',','') # replace commas with nothing
            numProps = int(numPropsText)
            numPages = 1 + (numProps - 1) // PROPS_PER_PAGE
            return numPages
        else: return 1        
    else:
        return -1

All interacitons with the Selenium Chrome driver are under the getPage function.

In [20]:
def getPage(URL, headless = False):
    if headless:
        HTML = requests.get(URL).content
        HTML = str(HTML)
    else:
        global driver
        try: driver.get(URL)
        except:
            print("Starting new driver.")
            driver = webdriver.Chrome(service = Service(executable_path = PATH))
                #driver = webdriver.Firefox(service = Service(executable_path = PATH))
        try: HTML = driver.page_source
        except:
            return getPage(URL, headless = headless)
        if len(HTML) < HTML_MIN_SIZE:
            driver.quit()
            sleep(WAIT_TIME('FAILURE',site))
            driver = webdriver.Chrome(service = Service(executable_path = PATH))
    if len(HTML) < HTML_MIN_SIZE:
        return getPage(URL, headless = headless)
    return HTML

In [21]:
def downloadZIP(site, ZIP, headless = False):
    filePath = generateFilePath(site, ZIP)
    if not isValidHTML(filePath):
        HTML = getPage(URL(site, ZIP), headless)
        HTML_to_file(HTML, filePath)
        
        numPages = parseNumPages(site, HTML)
        if not (numPages == -1):
            if numPages > 1:
                for page in range(2,numPages+1):
                    page = str(page)
                    filePath = generateFilePath(site, ZIP, page = page)
                    HTML = getPage(URL(site, ZIP, page), headless)
                    HTML_to_file(HTML, filePath)
        
        sleep(WAIT_TIME('CONSTANT',site))

In [ ]:
driver = webdriver.Chrome(service = Service(executable_path = PATH))
    #driver = webdriver.Firefox(service = Service(executable_path = PATH))
while True:
    for site in SITES_TO_SEARCH:
        for ZIP in tqdm(ZIPs):
            downloadZIP(site, ZIP)

  1%|▏         | 611/41070 [07:44<46:03:45,  4.10s/it]

In [ ]:
driver.quit()